In [1]:
import sys
sys.path.insert(1, '../..')

import torch
import torch.nn as nn
import random
import pandas as pd
import numpy as np

from library.evaluation import ConfusionMatrix

random.seed(33)

dataset_name = "Phemernr2_CrossVal"
unique_name = "T5"

In [2]:
vectors = np.loadtxt("../../data/processed/vectors/Phemernr2_T5_vectors.txt", delimiter=",")
vectors.shape

(6425, 768)

In [3]:
data = pd.read_csv("../../data/processed/phemernr2_dataset_with_tvt.csv", lineterminator="\n")
data.head()

,tweet_id,tweet_text,label,label2,topic,tvt,cv_fold,tt
0,552833795142209536,The East London Mosque would like to offer its...,non-rumours,non-rumours,charliehebdo-all-rnr-threads,test,2,test
1,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,rumours,true,germanwings-crash-all-rnr-threads,training,3,training
2,552798891994009601,Reports that two of the dead in the #CharlieHe...,rumours,true,charliehebdo-all-rnr-threads,test,2,test
3,576790814942236672,After #Putin disappeared Russian TV no longer ...,non-rumours,non-rumours,putinmissing-all-rnr-threads,test,2,test
4,499678822598340608,Saw #Ferguson for myself. #justiceformichaelbr...,non-rumours,non-rumours,ferguson-all-rnr-threads,training,3,training


In [4]:
labels_str = data['label2'].unique().tolist()
labels_str

['non-rumours', 'true', 'unverified', 'false']

In [5]:
labels = []
for i, d in data.iterrows():
    lab = labels_str.index(d['label2'])
#     labels.append([1 if j == lab else 0 for j in range(len(labels_str))])
    labels.append(lab)
labels[:10]

[0, 1, 1, 0, 0, 0, 0, 2, 0, 0]

In [6]:
test_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] == 0])
test_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] == 0])

In [7]:
print(test_vectors.shape)
print(test_labels.shape)

(630, 768)
(630,)


In [8]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import os
from typing import Callable


class NNClassifier(nn.Module):
    def __init__(self,
        n_input: int,
        n_output: int = 1,
        criterion: Callable = nn.BCELoss,
        beta1: float = 0.5,
        lr: float = 0.0002,
        device: str = None
    ):
        super(NNClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(n_input, 512),
            nn.LeakyReLU(0.1),
#             nn.BatchNorm1d(512),
            nn.Linear(512, 512),
            nn.LeakyReLU(0.1),
#             nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.1),
#             nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.1),
#             nn.BatchNorm1d(128),
            nn.Linear(128, n_output),
#             nn.Sigmoid()
        )
        self.criterion = criterion()
        if not device or device not in ['cpu', 'cuda']:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            self.device = device
        
        self.model = self.model.to(self.device)
        if self.device == 'cuda':
            self.model = torch.nn.DataParallel(self.model)
            cudnn.benchmark = True

        self.optimizer = optim.Adam(self.model.parameters(), lr=lr, betas=(beta1, 0.999))

    def forward(self, input):
        return self.model(input)
    
    def load_pretrained(self, filepath: str, key: str = "net", is_parallel: bool = False):
        checkpoint = torch.load(filepath)
        if is_parallel:
            self.model = torch.nn.DataParallel(self.model)
        self.model.load_state_dict(checkpoint[key], strict=False)
    
    def train_eval(self,
        train_x, train_y,
        test_x, test_y,
        n_iter: int = 100,
        batch_size: int = 128,
        saves: str = None
    ):
        trainset = torch.utils.data.TensorDataset(train_x, train_y) # create your datset
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size) # create your dataloader

        testset = torch.utils.data.TensorDataset(test_x, test_y) # create your datset
        testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size) # create your dataloader

        train_accs = []
        train_losses = []
        test_accs = []
        test_losses = []

        print(f"Using {self.device}")
        best_acc = 0
        best_loss = 1000
        best_test_acc = 0
        epoch = 0
        start_time = time.time()
        results = {}
        while True:
            epoch += 1
            self.model.train()
            train_loss = 0
            correct = 0
            total = 0
            for batch_idx, (inputs, targets) in enumerate(trainloader):
                self.model.zero_grad()
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                outputs = self.model(inputs)

                try:
                    loss = self.criterion(outputs, targets)
                except Exception:
                    loss = self.criterion(outputs, targets.long())

                loss.backward()
                self.optimizer.step()

                train_loss += loss.item()
                total += targets.size(0)
                
            train_losses.append(train_loss)

            self.model.eval()
            test_loss = 0
            test_acc = 0
            with torch.no_grad():
                inputs, targets = test_x.to(self.device), test_y.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets.long())

                test_loss += loss.item()
                
                preds = self.predict(test_x)
                conf_mat = ConfusionMatrix(
                    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_y]),
                    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds.cpu().numpy()]),
                    binary=False
                )
                conf_mat.evaluate(logs=False)
                test_acc = conf_mat.accuracy

            test_losses.append(test_loss)
            
#             if (epoch) % round(n_iter/20) == 0:
#                 print(f"-- Epoch {epoch}, Train Loss : {train_loss}, Test Loss : {test_loss}")

            # Save checkpoint.
#             if saves and test_loss < best_loss:
#                 print(f"Saving after new best loss : {test_loss}")
#                 best_loss = test_loss
            if saves and test_acc > best_test_acc:
#                 print(f"Saving after new best accuracy : {test_acc}")
                best_test_acc = test_acc

                state = {
                    'net': self.model.state_dict(),
                }
                if not os.path.isdir('models'):
                    os.mkdir('models')
                torch.save(state, f"../../data/models/{saves}.pth")
            
            if epoch >= n_iter:
                break

#         # visualizing accuracy over epoch
#         fig, ax2 = plt.subplots(1)
#         plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=1.5, wspace=0.4)

#         ax2.plot([i for i in range(len(train_losses))], train_losses, c='b', marker="o", label='Train Loss')
#         ax2.plot([i for i in range(len(test_losses))], test_losses, c='r', marker="o", label='Test Loss')
#         ax2.set_ylabel('Loss')
#         ax2.set_xlabel('Epoch')
#         ax2.set_xlim(0, len(train_losses))
#         ax2.set_ylim(min([min(train_losses), min(test_losses)])*0.1, max([max(train_losses), max(test_losses)]))
#         ax2.title.set_text(f"Loss over time (epoch)")
#         ax2.legend(loc='lower right')

#         plt.show()
    
    def predict(self, input_x):
        self.model.eval()
        with torch.no_grad():
            preds = self.model(torch.Tensor(input_x))
            preds = torch.log_softmax(preds, dim = 1)
            _, preds = torch.max(preds, dim = 1)
            return preds

In [9]:
print("3-Fold Cross Validation with 4-Layer Linear Network")

folds = [1, 2, 3]
for fold in folds:
    val_folds = [fold]
    train_folds = folds.copy()
    train_folds.remove(fold)

    train_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] in train_folds])
    val_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] in val_folds])

    train_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] in train_folds])
    val_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] in val_folds])

    print(f"Fold-{fold} Cross Validation")
    model_name = f"{dataset_name}_4LayerNet_{unique_name}_fold-{fold}"
    model = NNClassifier(train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss)
    model.train_eval(torch.Tensor(train_vectors),
                    torch.Tensor(train_labels),
                    torch.Tensor(val_vectors),
                    torch.Tensor(val_labels),
                    saves=model_name,
                    n_iter=1000,
                    batch_size=512)

    model.load_pretrained(f"../../data/models/{model_name}.pth")

    print(f"\n-------- Fold-{fold} Results --------")
    preds = model.predict(val_vectors)

    preds = preds.cpu().numpy()

    conf_mat = ConfusionMatrix(
        labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in val_labels]),
        predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
        binary=False,
        model_name=model_name
    )
    conf_mat.evaluate(classes=labels_str)
    print(f"-------- Fold-{fold} End --------\n")

3-Fold Cross Validation with 4-Layer Linear Network
Fold-1 Cross Validation
Using cuda

-------- Fold-1 Results --------
1914 vs 1914
Multi Class Evaluation

Class non-rumours Evaluation
- Precision : 84.558 %
- Recall : 92.007 %
- F1 : 0.88125

Class true Evaluation
- Precision : 65.723 %
- Recall : 64.11 %
- F1 : 0.64907

Class unverified Evaluation
- Precision : 65.789 %
- Recall : 40.107 %
- F1 : 0.49834

Class false Evaluation
- Precision : 71.622 %
- Recall : 60.571 %
- F1 : 0.65635

Combined Evaluation
- Accuracy : 79.31 %
- Precision : 71.923 %
- Recall : 64.199 %
- F1 : 0.67842

- Average Confidence : 100.0 %
Model, Combined,,,,non-rumours,,,true,,,unverified,,,false,,,
Phemernr2_CrossVal_4LayerNet_T5_fold-1, 79.31, 71.923, 64.199, 0.67842, 84.558, 92.007, 0.88125, 65.723, 64.11, 0.64907, 65.789, 40.107, 0.49834, 71.622, 60.571, 0.65635, 
-------- Fold-1 End --------

Fold-2 Cross Validation
Using cuda

-------- Fold-2 Results --------
1920 vs 1920
Multi Class Evaluation

Clas

In [10]:
fold_n = 1
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_4LayerNet_{unique_name}_fold-{fold_n}"
model = NNClassifier(train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss)

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors)

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-1

-------- Testing Results --------
630 vs 630
Multi Class Evaluation

Class non-rumours Evaluation
- Precision : 80.186 %
- Recall : 90.765 %
- F1 : 0.85149

Class true Evaluation
- Precision : 65.455 %
- Recall : 66.055 %
- F1 : 0.65753

Class unverified Evaluation
- Precision : 70.0 %
- Recall : 37.333 %
- F1 : 0.48696

Class false Evaluation
- Precision : 72.549 %
- Recall : 55.224 %
- F1 : 0.62712

Combined Evaluation
- Accuracy : 76.349 %
- Precision : 72.048 %
- Recall : 62.344 %
- F1 : 0.66846

- Average Confidence : 100.0 %
Model, Combined,,,,non-rumours,,,true,,,unverified,,,false,,,
Phemernr2_CrossVal_4LayerNet_T5_fold-1, 76.349, 72.048, 62.344, 0.66846, 80.186, 90.765, 0.85149, 65.455, 66.055, 0.65753, 70.0, 37.333, 0.48696, 72.549, 55.224, 0.62712, 
-------- Testing End --------



In [11]:
fold_n = 2
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_4LayerNet_{unique_name}_fold-{fold_n}"
model = NNClassifier(train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss)

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors)

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-2

-------- Testing Results --------
630 vs 630
Multi Class Evaluation

Class non-rumours Evaluation
- Precision : 81.499 %
- Recall : 91.821 %
- F1 : 0.86352

Class true Evaluation
- Precision : 67.29 %
- Recall : 66.055 %
- F1 : 0.66667

Class unverified Evaluation
- Precision : 63.83 %
- Recall : 40.0 %
- F1 : 0.4918

Class false Evaluation
- Precision : 75.51 %
- Recall : 55.224 %
- F1 : 0.63793

Combined Evaluation
- Accuracy : 77.302 %
- Precision : 72.032 %
- Recall : 63.275 %
- F1 : 0.6737

- Average Confidence : 100.0 %
Model, Combined,,,,non-rumours,,,true,,,unverified,,,false,,,
Phemernr2_CrossVal_4LayerNet_T5_fold-2, 77.302, 72.032, 63.275, 0.6737, 81.499, 91.821, 0.86352, 67.29, 66.055, 0.66667, 63.83, 40.0, 0.4918, 75.51, 55.224, 0.63793, 
-------- Testing End --------



In [12]:
fold_n = 3
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_4LayerNet_{unique_name}_fold-{fold_n}"
model = NNClassifier(train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss)

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors)

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-3

-------- Testing Results --------
630 vs 630
Multi Class Evaluation

Class non-rumours Evaluation
- Precision : 83.415 %
- Recall : 90.237 %
- F1 : 0.86692

Class true Evaluation
- Precision : 66.667 %
- Recall : 69.725 %
- F1 : 0.68161

Class unverified Evaluation
- Precision : 61.404 %
- Recall : 46.667 %
- F1 : 0.5303

Class false Evaluation
- Precision : 77.551 %
- Recall : 56.716 %
- F1 : 0.65517

Combined Evaluation
- Accuracy : 77.937 %
- Precision : 72.259 %
- Recall : 65.836 %
- F1 : 0.68898

- Average Confidence : 100.0 %
Model, Combined,,,,non-rumours,,,true,,,unverified,,,false,,,
Phemernr2_CrossVal_4LayerNet_T5_fold-3, 77.937, 72.259, 65.836, 0.68898, 83.415, 90.237, 0.86692, 66.667, 69.725, 0.68161, 61.404, 46.667, 0.5303, 77.551, 56.716, 0.65517, 
-------- Testing End --------

